## "Active Sciencing" with Reusable Workflows

By Kyle Cranmer & Lukas Heinrich June 4, 2016

Based on earlier work with NYU CDS masters students Manoj Kumar, Phil Yeres, and Michele Ceru and discussions with Brenden Lake and Gilles Louppe.

Define:

 1) $\phi$ : Experimental configuration

 2) $\theta$: Parameters that we would like to infer from the experimental data

 3) $X$ : Data generated from the experiment or simulator



In [ ]:
import numpy as np

## Step 1: Perform Experiment, Collect Data

In [ ]:
def collect_real_data(phi):
    '''External workflow performs experiment, collects data'''
    #secretly we will use the simulator with phi=phi_true
    pass

## Step 2: Bayesian Prior → Posterior Update 

In [ ]:
def calculate_posterior(prior, data, phi):
    '''External workflow runs simulator and performs likelihood free inference'''
    #this makes call to external workflow as a service that provides likelihood
    
    #keep this short. Import a module that will run emcee. Workflow provides likelihood
    pass

## Step 3: Optimize Experimental Configuration

Based on the updated posterior $p(\theta)$ we will consider future experiments with configuration $\phi$. For each of those configurations, we will run several simulations of the experiment and perform inference on those simulated datasets to estimate the expected information gain (EIG)

\begin{equation}
EIG(\phi) =  \int dx d\theta \; p(x | \theta) p(\theta) \big [ H\left [P(\theta) \right] - H\left[ P(\theta\, |\, x) \right] \big ] \approx \int dx  \; p(x | \theta_{MAP}) \big [ H\left [P(\theta) \right] - H\left[ P(\theta\, |\, x) \right] \big ]
\end{equation}
where
\begin{equation}
H\left [P(\theta) \right] = \int P(\theta) \log P(\theta) d\theta 
\end{equation}

To efficiently optimize $EIG[\phi]$ we will use an active learning procedure like Bayesian Optimization.

In [ ]:
def expected_information_gain(phi, prior):
    'calculate the expression above using workflow for simulations'
    n_simulations = 10
    
    #need to pass in prior through some extra arguments
    
    # use saddle-point approximation
    theta_map = prior.map()
    
    eig = np.zeros(n_simulations)
    
    for i_sim in range(n_simulations):
        # external workflow provides simulated data
        sim_data = collect_simulated_data(phi, theta_map)

        #external workflow uses simulator to provide likelihood 
        sim_posterior = calculate_posterior(prior, sim_data, phi)
        eig[i_sim] = info_gain(prior, sim_posterior)
        
    #check for outliers?
    
    return np.mean(eig)

In [ ]:
#use scikit-optimize to optimize phi
from skopt import gp_minimize
from skopt.plots import plot_evaluations

def design_next_experiment(prior):
    bounds = [(-5.0, 10.0), (0.0, 15.0)]
    n_calls = 50

    opt_result = gp_minimize(expected_information_gain, bounds, \
                             n_calls=n_calls, random_state=4)

    _ = plot_evaluations(opt_result, bins=10)
    return opt_phi

## Run the loop

### Initialize

In [ ]:
phi = 0.
prior_theta = None #simple class for range, density for emcee via KDE or histogram, MAP?

n_science_iterations = 10

phi_history = []
prior_history = []

for i_experiment in range(science_iterations):
    phi_history.append(phi)
    prior_history.append(prior)

    # run experiment with configuration given by phi
    real_data = collect_real_data(phi)

    #update new prior = posterior from previous experiment
    prior = calculate_posterior(prior, real_data, phi)
    
    #design new experiment given current knowledge
    phi=design_next_experiment(prior)
    

#make some plots of prior, and phi 

In [ ]:
#put this in a separate file
class prior_dist(self):

    '''
    member vars
        variable names
        ranges  (for use with george)
    methods
        density estimate (KDE or histogram) (for use with emcee)
        MAP (for saddle point approximation)
        
    '''